In [2]:
import os
import numpy as np
import cv2
os.chdir('C:/labels/train')
os.listdir(os.getcwd())


['IMG_MM000XXX_2021-05-12_18-11-22.346_6668.tif',
 'IMG_MM000XXX_2021-05-12_18-11-22.346_6668.tiff',
 'IMG_MM000XXX_2021-05-12_18-11-22.346_6668_parking.tiff',
 'IMG_MM000XXX_2021-05-12_18-11-22.346_6668_road.tif',
 'IMG_MM000XXX_2021-05-12_18-11-22.346_6668_service.tiff',
 'IMG_MM000XXX_2021-05-12_18-14-15.576_6684.tif',
 'IMG_MM000XXX_2021-05-12_18-14-15.576_6684.tiff',
 'IMG_MM000XXX_2021-05-12_18-14-15.576_6684_parking.tiff',
 'IMG_MM000XXX_2021-05-12_18-14-15.576_6684_road.tif',
 'IMG_MM000XXX_2021-05-12_18-14-15.576_6684_service.tiff',
 'IMG_MM000XXX_2021-05-12_18-19-44.649_6694.tif',
 'IMG_MM000XXX_2021-05-12_18-19-44.649_6694.tiff',
 'IMG_MM000XXX_2021-05-12_18-19-44.649_6694_parking.tiff',
 'IMG_MM000XXX_2021-05-12_18-19-44.649_6694_road.tif',
 'IMG_MM000XXX_2021-05-12_18-19-44.649_6694_service.tiff',
 'IMG_MM000XXX_2021-05-12_18-20-24.470_6701.tif',
 'IMG_MM000XXX_2021-05-12_18-20-24.470_6701.tiff',
 'IMG_MM000XXX_2021-05-12_18-20-24.470_6701_parking.tif',
 'IMG_MM000XXX_2021

In [2]:
os.chdir('C:/labels')

In [ ]:
def find_unique(master,sub):
    
    unique=[]
    #for i,m in enumerate(master):
    #    master[i]=master[i].split(".")[0]
    #master=master
    inst1=""
    for i,m in enumerate([m[:41] for m in master]):
        #print("==========")
        #print(m)
        is_unique=True
        for j,s in enumerate([s[:41] for s in sub]):
            #print("----------")
            #if
            #print("/t"+s)
            if m in s:
                is_unique=False
                break
        if is_unique:
            unique.append(master[i])
    return unique
infer=find_unique(os.listdir(),os.listdir('C:/labels/m'))

In [3]:
def p1_to_deepglobe(path):
    """
    Given path to a phaseone image
    Return list of 1024x1024 crops
    """
    #path="C:/ausm/phaseone/labels/IMG_MM000XXX_2021-05-12_18-10-35.997_6661.tif"
    img=cv2.imread(path)
    S=0.1442047376 # scale factor to convert phaseone GSD to DeepGlobe GSD 
    img=cv2.resize(img,None,fx=S,fy=S)
    return [img[:1024,:1024],img[:1024,-1024:],img[-1024:,:1024],img[-1024:,-1024:]]

def resize_crop_train_test(inpath,outpath):
    """
    Given inpath to directory with test/train folder system
    Return test/train folders in outpath directory
    """
    trainmode=False
    try:
        os.mkdir(outpath)
    except:
        pass
    #files=os.listdir(path)
    #print(files)
    #files=list(filter(lambda x: x.endswith('.tif') or x.endswith('.tiff'),files))
    #files=list(filter(lambda x: x.endswith('.tiff'),files))
    for d in ['test','train']:
        try:
            os.mkdir(os.path.join(outpath,d))
        except:
            pass
        
        path=os.path.join(inpath,d)
        files=os.listdir(path)
        count=0
        excl_count=0
        for file in files:
            (pre,sep,suf)=file.rpartition('.')
            #print(pre,sep,suf)
            if pre[-4:].isnumeric() and suf=='tif':
                #print(pre)
                img=p1_to_deepglobe(os.path.join(path,file))
                lab=p1_to_deepglobe(os.path.join(path,pre+'_road.tif'))
                count+=1
                for i,r in enumerate(lab):
                    if np.sum(r)>0 or not trainmode:
                        #print(os.path.join(path,'resize','RSZ_'+pre+'_'+str(i)+sep+suf))
                        cv2.imwrite(os.path.join(outpath,d,'RSZ_'+pre+'_'+str(i)+'.tif'),img[i]) 
                        cv2.imwrite(os.path.join(outpath,d,'RSZ_'+pre+'_'+str(i)+'.tiff'),r)
                    else:
                        excl_count+=1
                        print(pre+'_'+str(i),f"\n{excl_count} label(s) with no roads detected and omitted")
        print(f"{excl_count} resized crops omitted")

def resize_crop_infer(inpath,outpath):
    #files=os.listdir(path)
    #print(files)
    #files=list(filter(lambda x: x.endswith('.tif') or x.endswith('.tiff'),files))
    #files=list(filter(lambda x: x.endswith('.tiff'),files))
    #path=os.path.join(inpath,d)
    files=os.listdir(inpath)
    #count=0
    #excl_count=0
    for file in files:
        (pre,sep,suf)=file.rpartition('.')
        if suf=='tif':
            print(pre)
            imgs=p1_to_deepglobe(os.path.join(inpath,file))
            for i,img in enumerate(imgs):
                cv2.imwrite(os.path.join(outpath,'RSZ_'+pre+'_'+str(i)+'.tif'),img)
        #(pre,sep,suf)=file.rpartition('.')
        #print(pre,sep,suf)
        #if pre[-4:].isnumeric() and suf=='tif':
            #print(pre)
            #img=p1_to_deepglobe(os.path.join(path,file))
            #lab=p1_to_deepglobe(os.path.join(path,pre+'_road.tif'))
            #count+=1
            #for i,r in enumerate(lab):
                #if np.sum(r)>0 or not trainmode:
                    #print(os.path.join(path,'resize','RSZ_'+pre+'_'+str(i)+sep+suf))
                    #cv2.imwrite(os.path.join(outpath,d,'RSZ_'+pre+'_'+str(i)+'.tif'),img[i]) 
                    #cv2.imwrite(os.path.join(outpath,d,'RSZ_'+pre+'_'+str(i)+'.tiff'),r)
                #else:
                    #excl_count+=1
                    #print(pre+'_'+str(i),f"\n{excl_count} label(s) with no roads detected and omitted")
        #print(r.shape)
        #break
resize_crop_infer('E:/infer/','E:/infer/rsz')

IMG_MM000XXX_2021-05-12_18-10-35.997_6661
IMG_MM000XXX_2021-05-12_18-10-42.342_6662
IMG_MM000XXX_2021-05-12_18-10-48.927_6663
IMG_MM000XXX_2021-05-12_18-10-55.544_6664
IMG_MM000XXX_2021-05-12_18-11-02.153_6665
IMG_MM000XXX_2021-05-12_18-11-08.759_6666
IMG_MM000XXX_2021-05-12_18-11-15.539_6667
IMG_MM000XXX_2021-05-12_18-11-22.346_6668
IMG_MM000XXX_2021-05-12_18-11-29.029_6669
IMG_MM000XXX_2021-05-12_18-11-35.345_6670
IMG_MM000XXX_2021-05-12_18-13-31.212_6676
IMG_MM000XXX_2021-05-12_18-13-36.849_6677
IMG_MM000XXX_2021-05-12_18-13-42.381_6678
IMG_MM000XXX_2021-05-12_18-13-48.049_6679
IMG_MM000XXX_2021-05-12_18-13-53.394_6680
IMG_MM000XXX_2021-05-12_18-13-58.793_6681
IMG_MM000XXX_2021-05-12_18-14-04.409_6682
IMG_MM000XXX_2021-05-12_18-14-10.000_6683
IMG_MM000XXX_2021-05-12_18-14-15.576_6684
IMG_MM000XXX_2021-05-12_18-14-21.114_6685
IMG_MM000XXX_2021-05-12_18-14-26.592_6686
IMG_MM000XXX_2021-05-12_18-14-32.181_6687
IMG_MM000XXX_2021-05-12_18-14-37.778_6688
IMG_MM000XXX_2021-05-12_18-14-43.5

IMG_MM000XXX_2021-05-12_18-58-30.931_6876
IMG_MM000XXX_2021-05-12_18-58-36.754_6877
IMG_MM000XXX_2021-05-12_18-58-42.548_6878
IMG_MM000XXX_2021-05-12_18-58-48.338_6879
IMG_MM000XXX_2021-05-12_19-01-11.555_6881
IMG_MM000XXX_2021-05-12_19-01-18.747_6882
IMG_MM000XXX_2021-05-12_19-01-25.939_6883
IMG_MM000XXX_2021-05-12_19-01-32.781_6884
IMG_MM000XXX_2021-05-12_19-01-39.589_6885
IMG_MM000XXX_2021-05-12_19-01-46.168_6886
IMG_MM000XXX_2021-05-12_19-01-52.772_6887
IMG_MM000XXX_2021-05-12_19-01-59.382_6888
IMG_MM000XXX_2021-05-12_19-02-06.189_6889
IMG_MM000XXX_2021-05-12_19-02-13.018_6890
IMG_MM000XXX_2021-05-12_19-02-19.944_6891
IMG_MM000XXX_2021-05-12_19-02-26.750_6892
IMG_MM000XXX_2021-05-12_19-02-33.635_6893
IMG_MM000XXX_2021-05-12_19-02-40.367_6894
IMG_MM000XXX_2021-05-12_19-02-47.395_6895
IMG_MM000XXX_2021-05-12_19-05-11.001_6900
IMG_MM000XXX_2021-05-12_19-05-17.388_6901
IMG_MM000XXX_2021-05-12_19-05-23.797_6902
IMG_MM000XXX_2021-05-12_19-05-30.219_6903
IMG_MM000XXX_2021-05-12_19-05-36.5

In [4]:
resize_crop_train_test('C:/labels','C:/labels/set1')

IMG_MM000XXX_2021-05-12_18-14-15.576_6684_2 
1 label(s) with no roads detected and omitted
IMG_MM000XXX_2021-05-12_18-44-55.571_6809_0 
2 label(s) with no roads detected and omitted
IMG_MM000XXX_2021-05-12_18-44-55.571_6809_2 
3 label(s) with no roads detected and omitted


In [14]:
import warnings

def merge_masks(inpath,outpath):
    """merge_masks(inpath):
    
        Merges 4 1024x1024 fuzzy masks into 10652x14204 binary masks from inpath to outpath
        
    """
    FUZZY=False
    EXT = 'fuzzy' if FUZZY else 'mask'
    #S=1/0.1442047376 # scale factor to convert phaseone GSD to DeepGlobe GSD 
    S=6.934584929 # scale factor to convert DeepGlobe GSD to phaseone GSD 
    
    overlap_arr=np.zeros([1536,2048])
    overlap_arr[:1024,:1024]+=1
    overlap_arr[:1024,-1024:]+=1
    overlap_arr[-1024:,:1024]+=1
    overlap_arr[-1024:,-1024:]+=1

    def myTile(file,tile):
        index=int(file.split('_')[-2])
        img=cv2.imread(os.path.join(inpath,file),0)
        if index == 0:
            tile[:1024,:1024]+=img
        elif index==1:
            tile[:1024,-1024:]+=img
        elif index==2:
            tile[-1024:,:1024]+=img
        elif index==3:
            tile[-1024:,-1024:]+=img
        return tile
    #try:
    #    os.mkdir(os.path.join(inpath,'merge'))
    #except:
    #    warnings.warn('/merge folder exists')
        
    isFirst=True
    img_num=0
    files=os.listdir(inpath)
    for file in files:
        if '.png' in file:
            img_num+=1
    print(img_num)

    

    count=0
    for i,file in enumerate(files):
        if '.png' in file:
            count+=1
            print(count)
            if isFirst:

                tile=np.zeros([1536,2048])
                curr_file=file.rsplit('_',2)[0]
                parent_file=curr_file
                tile=myTile(file,tile)
                isFirst=False
            else:
                curr_file=file.rsplit('_',2)[0]
                if curr_file == parent_file:
                    tile=myTile(file,tile)
                else:
                    tile=np.divide(tile,overlap_arr)                    
                    tile=cv2.resize(tile,(14204,10652),interpolation=cv2.INTER_LINEAR)
                    
                    if not FUZZY:
                        tile[tile>=128]=255
                        tile[tile<128]=0
                        
                    cv2.imwrite(os.path.join(outpath,parent_file.split('_',1)[1]+'_'+EXT+'.png'),
                                tile.astype(np.uint8))
                    
                    tile=np.zeros([1536,2048])
                    tile=myTile(file,tile)
                    
                    parent_file=curr_file
                    
                if count == img_num:
                    tile=np.divide(tile,overlap_arr)              
                    tile=cv2.resize(tile,(14204,10652),interpolation=cv2.INTER_LINEAR)
                    
                    if not FUZZY:                        
                        tile[tile>=128]=255
                        tile[tile<128]=0
                        
                    cv2.imwrite(os.path.join(outpath,parent_file.split('_',1)[1]+'_'+EXT+'.png'),
                                tile.astype(np.uint8))
        #break
                
merge_masks('C:/labels/out/first02_merge','C:/labels/out/finetune')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [2]:
#def crops_to_phaseone(path):
#    #src="Something unique"
##    crops = os.listdir(path)
 ####   isFirst = True
 #   isRoot = True
 #   for (root, dirs, files) in os.walk(path, topdown=True):
 #       if isRoot:
 #           for i, crop in enumerate(files):
 #               curr_file = crop[:41]
 #               # print(crop)
 #               if isFirst:
 ####                   # print(len(crops))
    #                isFirst = False
    #                tile = np.zeros((10652, 14204, 3), dtype=np.uint8)
     #               index = int(crop.split('_')[-2].split('_')[-1])
    #                row = math.floor(index/4)
      #              col = index-(row)*4
       #             tile[row*2663:(row+1)*2663, col*3551:(col+1) *
        #                 3551, :] = cv2.imread(path+crop)
         #           parent_file = crop[:41]
          #          #print("New source image detected:", src)
           #     if curr_file == parent_file:
            ##       row = math.floor(index/4)
              #      col = index-(row)*4
               #     tile[row*2663:(row+1)*2663, col*3551:(col+1) *
#                         3551, :] = cv2.imread(path+crop)
#                else:
#                    cv2.imwrite("C:/ausm/phaseone/labels/" +
#                                parent_file+".tiff", tile)
#                    parent_file = curr_file
#                    tile = np.zeros((10652, 14204, 3), dtype=np.uint8)
#                    index = int(crop.split('_')[-2].split('_')[-1])
#                    row = math.floor(index/4)
#                    col = index-(row)*4
##                    tile[row*2663:(row+1)*2663, col*3551:(col+1) *
#                         3551, :] = cv2.imread(path+crop)
#                if i == len(files)-1:
#                    cv2.imwrite("C:/ausm/phaseone/labels/" +
#                                parent_file+".tiff", tile)
#        isRoot = False
#        break

In [17]:
fuck='RSZ_IMG_MM000XXX_2021-05-12_18-10-35.997_6661_0_mask.png'
index=int(fuck.split('_')[-2])
index
prefix=fuck.rsplit('_',2)[0]
prefix
if index==0 

'RSZ_IMG_MM000XXX_2021-05-12_18-10-35.997_6661'

In [27]:
fuck=np.zeros([4,4])
fuck=fuck==0
fuck

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])